Setup

In [1]:
import torch
print(torch.__version__)

1.12.1+cu102


In [2]:
!export PATH=/usr/local/cuda-11.1/bin:$PATH

In [3]:
!pip uninstall torch-scatter --yes 
!pip uninstall torch-sparse --yes
!pip uninstall torch-cluster --yes
!pip uninstall torch-spline-conv --yes
!pip uninstall torch-geometric --yes
!pip uninstall glycowork --y

Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Successfully uninstalled torch-scatter-2.0.9
Found existing installation: torch-sparse 0.6.15
Uninstalling torch-sparse-0.6.15:
  Successfully uninstalled torch-sparse-0.6.15
Found existing installation: torch-cluster 1.6.0
Uninstalling torch-cluster-1.6.0:
  Successfully uninstalled torch-cluster-1.6.0
Found existing installation: torch-spline-conv 1.2.1
Uninstalling torch-spline-conv-1.2.1:
  Successfully uninstalled torch-spline-conv-1.2.1
Found existing installation: torch-geometric 2.1.0.post1
Uninstalling torch-geometric-2.1.0.post1:
  Successfully uninstalled torch-geometric-2.1.0.post1
Found existing installation: glycowork 0.5.0
Uninstalling glycowork-0.5.0:
  Successfully uninstalled glycowork-0.5.0


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-sparse --no-cache-dir -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install glycowork

In [5]:
import glycowork
#export
from glycowork.ml.model_training import *
from glycowork.ml.models import *
from glycowork.ml.processing import *
from glycowork.ml.inference import *
from glycowork.ml.train_test_split import *
from glycowork.ml import models
from glycowork.ml import model_training
#export
from glycowork.glycan_data.loader import *
from glycowork.glycan_data.data_entry import *
from glycowork.motif.analysis import plot_embeddings, make_heatmap, characterize_monosaccharide, get_pvals_motifs

In [6]:
!pip install nbdev
#hide
#default_exp ml
import warnings
warnings.filterwarnings("ignore")
from nbdev.showdoc import show_doc
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

In [7]:
!pip install fair-esm
import esm
model_esm, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

Parse Protein Input

In [8]:
#CHIEMRA INPUT
!gsutil ls gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork
!gsutil cp gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10GBChimera.output chimeras.output
protein_db = []

gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10GBChimera.output
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/stored_protein_embeddings.pkl
Copying gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10GBChimera.output...
/ [1 files][994.3 MiB/994.3 MiB]   59.5 MiB/s                                   
Operation completed over 1 objects/994.3 MiB.                                    


In [24]:
file_input = pd.read_csv('chimeras.output', nrows = 100, header = None)
file_input.columns = ['input']

In [25]:
file_input = file_input.input.str.split(expand = True)

In [241]:
#file_input.head()

In [27]:
proteinNameSeq = file_input.drop([1,2], axis = 1)
proteinNameSeq.columns = ['NCR', 'SEQUENCE']

In [28]:
proteinNameSeq.head()
#for index, row in proteinNameSeq.iterrows():
    #protein_embedding.append(row['SEQUENCE'])
protein_embeddings = []
protein_embeddings

[]

In [ ]:
#DEFAULT FOR GLYCOWORK
#import os
#bucket_root = os.environ['WORKSPACE_BUCKET']
#print(bucket_root)
## find out the path to file by running gsutil commandline tool.
#!gsutil ls gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork
#file1 = 'stored_protein_embeddings.pkl'

#!gsutil cp gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/stored_protein_embeddings.pkl stored_protein_embeddings.pkl

#with open('stored_protein_embeddings.pkl', 'rb') as file1:
#  old_dic = pickle.load(file)

Input for Embeddings

In [ ]:
#show_doc(get_esm1b_representations)

In [ ]:
#represent = get_esm1b_representations(lane, model_esm, alphabet)

In [29]:
protein_seq = proteinNameSeq['SEQUENCE'].tolist()
protein_dict = get_esm1b_representations(protein_seq, model_esm, alphabet)

In [239]:
#proteinNameSeq.head()

Code

In [17]:
model = prep_model('LectinOracle',1,trained=True)

In [178]:
outprint_multi_protein = pd.DataFrame(columns = ['name', 'preds'])

In [110]:
#lane sequence
glycan = ['GlcA(b1-3)Xyl(a1-3)GlcA(b1-3)Xyl']

In [148]:
#show_doc(get_lectin_preds)

In [179]:
#multiple protein, single glycan
a=0
for index, rows in proteinNameSeq.iterrows():
    outprint_multi_protein.at[a, 'name'] = rows['NCR'] # replace with the actual name when reading from a file
    #[pred] is at the end because get_lec returns an array and we only need the pred value
    outprint_multi_protein.at[a, 'preds'] = str((get_lectin_preds((rows['SEQUENCE']), glycan,model,protein_dict))['pred'])
    a +=1
#outprint_multi_protein.head()

In [240]:
nameDF = outprint_multi_protein['name']
splitDF = outprint_multi_protein.preds.str.split(expand = True)
splitDF = splitDF.drop([0,2,3,4,5], axis = 1)
concatDF = pd.concat([nameDF, splitDF], ignore_index=True, sort=False, axis = 1)
#concatDF

In [238]:
#PREDICTION OUTPUT CSV
outprint_multi_protein_3.to_csv("output.csv",header=False, index = False)
!gsutil ls gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork
!gsutil cp "output.csv" gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/output.csv

gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10GBChimera.output
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/output.csv
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/stored_protein_embeddings.pkl
Copying file://output.csv [Content-Type=text/csv]...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


In [ ]:
# single protein, multiple glycan
#preds = get_lectin_preds(protein, df_glycan.glycan.tolist(),model,old_dic)
preds = get_lectin_preds(lane, df_glycan.glycan.tolist(),model,represent)
print(preds.head())

In [ ]:
# single protein, single glycan
preds = get_lectin_preds(protein_SNA, glycan,model,old_dic)
print(preds.head())